In [2]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive',force_remount=True)


Mounted at /content/gdrive


# Importing the required Libraries

In [3]:
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout,BatchNormalization
from keras import backend as K
import keras
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential, Model
from keras.layers import MaxPool2D
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score,accuracy_score
from collections import Counter

# Creating the Dataset

In [4]:
def create_dataset(img_folder):
   
    image_path=[]
    labels=[]
       
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):       
            image_path.append(os.path.join(img_folder, dir1,file))
            label = dir1
            labels.append(label)
       
    return image_path,labels

# Preprocessing The Dataset

In [5]:
def preprocess(X):
  preprocess_X = []
  for x in X:
    image = load_img(x, target_size=(300, 300))
    image = img_to_array(image)
    image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    preprocess_X.append(image)
  
  return preprocess_X

In [6]:
X,Y= create_dataset('/content/gdrive/MyDrive/Final Project/TrashNet/dataset-resized')

In [7]:
le = LabelEncoder()
le.fit(Y)
Y = le.transform(Y)

In [8]:
X = preprocess(X)

In [9]:
X = np.array(X)
# Y = np.array(Y)

# Splitting the Dataset

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle = True)

# Model Architectures

## VGG16 Model

In [11]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(6, activation='softmax', name='output'))
    return model


# Model Inspired By VGG16 Model

In [12]:
def modelVGG():
  model = Sequential()
  model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten(name='flatten'))
  model.add(Dense(128, activation='relu', name='fc1'))
  model.add(Dense(6, activation='softmax', name='output'))
  return model



# Model Inspired By VGG16 Model with Dropout

In [14]:
def modelVGG_Dropout():
  model = Sequential()
  model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten(name='flatten'))
  model.add(Dropout(0.6))
  model.add(Dense(128, activation='relu', name='fc1'))
  model.add(Dense(6, activation='softmax', name='output'))
  return model



# VGG16 inspired model with Batch Normalisation and Dropout Regularisation Techniques

In [13]:
def modelVGG_BN():
  model = Sequential()
  model.add(Conv2D(input_shape=(224,224,3),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten(name='flatten'))
  model.add(BatchNormalization())
  model.add(Dropout(0.6))
  model.add(Dense(128, activation='relu', name='fc1'))
  model.add(Dense(6, activation='softmax', name='output'))
  return model



# Model 2 Architecture

In [12]:
def model2():
  model=Sequential()
  #Convolution blocks

  model.add(Conv2D(256,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
  model.add(MaxPool2D(pool_size=2)) 
  model.add(BatchNormalization())

  model.add(Conv2D(128,(3,3), padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=2)) 
  model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=2)) 
  model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=2)) 

  #Classification layers
  model.add(Flatten())
  model.add(Dense(64,activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(32,activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(6,activation='softmax'))
  return model


# Training VGG model 

In [15]:
vgg_model = VGG16()

In [16]:
opt = SGD(learning_rate=1e-3, momentum=0.9)
vgg_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])

In [18]:
vgg_model.fit(X_train,Y_train,epochs=50,verbose=1)

Epoch 1/50
56/56 [==============================] - 24s 429ms/step - loss: 1.6502 - accuracy: 0.3003
Epoch 2/50
56/56 [==============================] - 21s 379ms/step - loss: 1.4486 - accuracy: 0.4197
Epoch 3/50
56/56 [==============================] - 21s 384ms/step - loss: 1.2535 - accuracy: 0.5011
Epoch 4/50
56/56 [==============================] - 22s 389ms/step - loss: 1.2146 - accuracy: 0.5226
Epoch 5/50
56/56 [==============================] - 22s 394ms/step - loss: 1.1141 - accuracy: 0.5718
Epoch 6/50
56/56 [==============================] - 22s 395ms/step - loss: 1.0511 - accuracy: 0.5979
Epoch 7/50
56/56 [==============================] - 22s 400ms/step - loss: 0.9647 - accuracy: 0.6346
Epoch 8/50
56/56 [==============================] - 23s 402ms/step - loss: 0.9085 - accuracy: 0.6533
Epoch 9/50
56/56 [==============================] - 23s 402ms/step - loss: 0.8299 - accuracy: 0.6951
Epoch 10/50
56/56 [==============================] - 23s 405ms/step - loss: 0.7827 - accura

In [19]:
Y_pred = vgg_model.predict(X_test)
Y_pred = tf.argmax(input=Y_pred, axis=1)
m = tf.keras.metrics.Accuracy()
m.update_state(Y_pred, Y_test)
m.result().numpy()

0.743083

# Training our base model inspired by VGG16

In [20]:
model2 = modelVGG()
opt = tf.optimizers.Adam(learning_rate= 0.0005)
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])
model2.fit(X_train,Y_train,epochs=50,verbose=1)

Epoch 1/50
56/56 [==============================] - 12s 152ms/step - loss: 5.0939 - accuracy: 0.3795
Epoch 2/50
56/56 [==============================] - 8s 134ms/step - loss: 1.2679 - accuracy: 0.5102
Epoch 3/50
56/56 [==============================] - 8s 135ms/step - loss: 1.0647 - accuracy: 0.5973
Epoch 4/50
56/56 [==============================] - 8s 135ms/step - loss: 0.8240 - accuracy: 0.6912
Epoch 5/50
56/56 [==============================] - 8s 136ms/step - loss: 0.5657 - accuracy: 0.8054
Epoch 6/50
56/56 [==============================] - 8s 135ms/step - loss: 0.3021 - accuracy: 0.9010
Epoch 7/50
56/56 [==============================] - 8s 136ms/step - loss: 0.2360 - accuracy: 0.9231
Epoch 8/50
56/56 [==============================] - 8s 136ms/step - loss: 0.2179 - accuracy: 0.9270
Epoch 9/50
56/56 [==============================] - 8s 136ms/step - loss: 0.1927 - accuracy: 0.9378
Epoch 10/50
56/56 [==============================] - 8s 137ms/step - loss: 0.0741 - accuracy: 0.980

In [21]:
Y_pred2 = model2.predict(X_test)
Y_pred2 = tf.argmax(input=Y_pred2, axis=1)
m2 = tf.keras.metrics.Accuracy()
m2.update_state(Y_pred2, Y_test)
m2.result().numpy()

0.54018444

# Experiments with our model including regularisation Techniques : DROPOUT

In [22]:
model3 = modelVGG_Dropout()
opt = tf.optimizers.Adam(learning_rate= 0.0005)
model3.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])
model3.fit(X_train,Y_train, epochs=50,verbose=1)
Y_pred3 = model3.predict(X_test)
Y_pred3 = tf.argmax(input=Y_pred3, axis=1)

Epoch 1/50
56/56 [==============================] - 8s 136ms/step - loss: 4.2471 - accuracy: 0.3416
Epoch 2/50
56/56 [==============================] - 8s 136ms/step - loss: 1.2594 - accuracy: 0.5119
Epoch 3/50
56/56 [==============================] - 8s 137ms/step - loss: 1.0868 - accuracy: 0.5916
Epoch 4/50
56/56 [==============================] - 8s 136ms/step - loss: 0.8984 - accuracy: 0.6691
Epoch 5/50
56/56 [==============================] - 8s 142ms/step - loss: 0.7409 - accuracy: 0.7291
Epoch 6/50
56/56 [==============================] - 8s 139ms/step - loss: 0.6240 - accuracy: 0.7738
Epoch 7/50
56/56 [==============================] - 8s 136ms/step - loss: 0.4142 - accuracy: 0.8558
Epoch 8/50
56/56 [==============================] - 8s 137ms/step - loss: 0.2814 - accuracy: 0.9005
Epoch 9/50
56/56 [==============================] - 8s 138ms/step - loss: 0.2430 - accuracy: 0.9152
Epoch 10/50
56/56 [==============================] - 8s 138ms/step - loss: 0.2270 - accuracy: 0.9219

In [23]:
m3 = tf.keras.metrics.Accuracy()
m3.update_state(Y_pred3, Y_test)
m3.result().numpy()

0.6482214

# Experiments with our model using regualisation technique: Batch Normalisation

In [24]:
model4 = modelVGG_BN()
opt = tf.optimizers.Adam(learning_rate= 0.0005)
model4.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])
model4.fit(X_train,Y_train, epochs=50,verbose=1)
Y_pred4 = model4.predict(X_test)
Y_pred4 = tf.argmax(input=Y_pred4, axis=1)

Epoch 1/50
56/56 [==============================] - 10s 134ms/step - loss: 4.8102 - accuracy: 0.4830
Epoch 2/50
56/56 [==============================] - 7s 127ms/step - loss: 2.7286 - accuracy: 0.6058
Epoch 3/50
56/56 [==============================] - 7s 128ms/step - loss: 1.6062 - accuracy: 0.6827
Epoch 4/50
56/56 [==============================] - 7s 127ms/step - loss: 1.0079 - accuracy: 0.7477
Epoch 5/50
56/56 [==============================] - 7s 128ms/step - loss: 0.9425 - accuracy: 0.8049
Epoch 6/50
56/56 [==============================] - 7s 128ms/step - loss: 0.6101 - accuracy: 0.8428
Epoch 7/50
56/56 [==============================] - 7s 128ms/step - loss: 0.5961 - accuracy: 0.8682
Epoch 8/50
56/56 [==============================] - 7s 128ms/step - loss: 0.5737 - accuracy: 0.8558
Epoch 9/50
56/56 [==============================] - 7s 129ms/step - loss: 0.4537 - accuracy: 0.8925
Epoch 10/50
56/56 [==============================] - 7s 129ms/step - loss: 0.4683 - accuracy: 0.900

In [25]:
m4 = tf.keras.metrics.Accuracy()
m4.update_state(Y_pred4, Y_test)
m4.result().numpy()

0.6627141

# Experiments with our model with Data Augmentation

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(brightness_range=(0.5, 0.99), zoom_range=[0.7, 1.3])
data_generator.fit(X, augment=True)
train_gen = data_generator.flow(x=X_train, y=Y_train)
test_gen = data_generator.flow(x=X_test, y=Y_test)

In [15]:
model5 = modelVGG_BN()
opt = tf.optimizers.Adam(learning_rate= 0.0005)
model5.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])
model5.fit(train_gen,validation_data=test_gen, epochs=50,verbose=1)
Y_pred5 = model5.predict(X_test)
Y_pred5 = tf.argmax(input=Y_pred5, axis=1)

Epoch 1/50
56/56 [==============================] - 47s 594ms/step - loss: 6.1802 - accuracy: 0.3908 - val_loss: 9.9676 - val_accuracy: 0.1963
Epoch 2/50
56/56 [==============================] - 31s 557ms/step - loss: 4.3732 - accuracy: 0.4564 - val_loss: 8.2749 - val_accuracy: 0.2701
Epoch 3/50
56/56 [==============================] - 32s 564ms/step - loss: 3.0289 - accuracy: 0.4745 - val_loss: 2.1121 - val_accuracy: 0.2793
Epoch 4/50
56/56 [==============================] - 32s 567ms/step - loss: 1.6719 - accuracy: 0.5402 - val_loss: 1.5173 - val_accuracy: 0.3518
Epoch 5/50
56/56 [==============================] - 31s 557ms/step - loss: 1.2816 - accuracy: 0.6069 - val_loss: 3.2602 - val_accuracy: 0.3821
Epoch 6/50
56/56 [==============================] - 35s 635ms/step - loss: 1.2151 - accuracy: 0.5865 - val_loss: 1.6600 - val_accuracy: 0.3570
Epoch 7/50
56/56 [==============================] - 40s 721ms/step - loss: 1.0498 - accuracy: 0.6290 - val_loss: 1.5200 - val_accuracy: 0.4480

# Training the Second Model

In [13]:
model6 =model2()
opt = tf.optimizers.Adam(learning_rate= 0.0005)
model6.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=opt,metrics=["accuracy"])
model6.fit(X_train,Y_train, epochs=50,verbose=1)
Y_pred6 = model6.predict(X_test)
Y_pred6 = tf.argmax(input=Y_pred6, axis=1)

Epoch 1/50
56/56 [==============================] - 38s 422ms/step - loss: 1.7815 - accuracy: 0.2308
Epoch 2/50
56/56 [==============================] - 21s 383ms/step - loss: 1.6200 - accuracy: 0.3162
Epoch 3/50
56/56 [==============================] - 21s 382ms/step - loss: 1.5599 - accuracy: 0.3722
Epoch 4/50
56/56 [==============================] - 22s 385ms/step - loss: 1.4630 - accuracy: 0.3937
Epoch 5/50
56/56 [==============================] - 22s 386ms/step - loss: 1.4127 - accuracy: 0.4576
Epoch 6/50
56/56 [==============================] - 22s 388ms/step - loss: 1.3256 - accuracy: 0.4859
Epoch 7/50
56/56 [==============================] - 22s 390ms/step - loss: 1.2518 - accuracy: 0.5317
Epoch 8/50
56/56 [==============================] - 22s 390ms/step - loss: 1.1781 - accuracy: 0.5520
Epoch 9/50
56/56 [==============================] - 22s 392ms/step - loss: 1.0730 - accuracy: 0.5962
Epoch 10/50
56/56 [==============================] - 22s 393ms/step - loss: 0.9867 - accura